# Single Leptoquark Analysis

In [ ]:
import os
import ROOT
from ROOT import *

In [ ]:
def getData (fullAnalysis=True):
    """
    Function to read data from Delphes-generated events
    """
    ev_dir = os.getcwd() + '/Events'    # Event directory path
    
    jobs = 1
    if fullAnalysis:
        root, dirs, files = os.walk(ev_dir)
        jobs = len(dirs)
    
    tree = TChain('Delphes;1')
    for i in range(jobs):
        path = str('Events/run_{:02d}/tag_1_delphes_events.root'.format(i))
        tree.Add(path)
        print('Run found in ' + path)
    print('Total events uploaded: {}'.format(tree.GetEntries()))    

In [ ]:
class JetVector:
    def __init__(self, event, j):
        self.TLV = TLorentzVector()
        self.TLV.SetPtEtaPhiM(event.GetLeaf('Jet.PT').GetValue(j),
                              event.GetLeaf('Jet.Eta').GetValue(j),
                              event.GetLeaf('Jet.Phi').GetValue(j),
                              event.GetLeaf('Jet.Mass').GetValue(j)
                             )
        self.BTag = event.GetLeaf('Jet.BTag').GetValue(j)
        self.TauTag = event.GetLeaf('Jet.TauTag').GetValue(j)
        
    
    ### Retrieval methods
        
    def GetTLV (self):    
        return self.TLV
    
    def GetBTag (self):
        return self.BTag
    
    def GetTauTag (self):
        return self.TauTag

In [ ]:
def PT(jet):
    # Returns jet transverse momentum
    TLV = jet.GetTLV()
    return TLV.Pt()


def Eta(jet):
    # Returns jet pseudorapidity
    TLV = jet.GetTLV()
    return TLV.Eta()


def getGoodJets(event):
    """
    Separate jet types into a map structure.
    """
    jetTypes = {'jets' : [],
                'b_jets' : [],
                'tau_jets' : [],
                'all_jets' : [],
                'other_jets' : []
               }
    
    n_jets = event.Jet.GetEntries()
    
    for j in range(n_jets):
        jet = JetVector(event, j)
        
        if PT(jet)<= 20.:
            continue
        
        jetTypes['all_jets'] += [jet]
        if (jet.GetBTag() == 0 and jet.GetBTag() == 0):
            if PT(jet)>30.:
                jetTypes['jets'] += [jet]
                
        elif (jet.GetBTag() == 1 and jet.GetBTag() == 0):
            if (PT(jet)>30.) and abs(Eta(jet))<2.5:
                jetTypes['b_jets'] += [jet]
        
        elif (jet.GetBTag() == 0 and jet.GetBTag() == 1):
            if abs(Eta(jet))<2.5:
                jetTypes['tau_jets'] += [jet]
        else:
            jetTypes['other_jets'] += [jet]
            
    for key in jetTypes:
        jetTypes[key].sort(reverse = True, key = PT)
        
    return jetTypes